In [1]:
from pysgx import info, stocks, util
import importlib
import pandas as pd
from typing import List
from dataclasses import dataclass
import numpy as np

In [7]:
import warnings

warnings.filterwarnings('ignore')

In [88]:
# Re-run this cell if pysgx module have hot changes
importlib.reload(info);
importlib.reload(stocks);
importlib.reload(util);

# Portfolio

In [2]:
all_stocks = stocks.loads(info.all_tickers)

In [12]:
period_start = util.YearMonth(2000, 1)
period_last = util.YearMonth(2023, 1)


def get_portfolio(tickers):
    portfolio = []
    for stock in all_stocks:
        if stock.ticker in tickers:
            portfolio.append(stock)
    return portfolio


ByAddBackClose = []
ByAdjClose = []

while period_start <= period_last:
    returns = dict()
    for stock in all_stocks:
        period_end = util.get_next(period_start, 12)
        adtv = stock.get_adtv(period_start, period_end)

        if adtv > 100000:
            r = stocks.get_return(period_start, period_end, stock, False)
            if not np.isnan(r.ByAddBackClose):
                returns[stock.ticker] = r.ByAddBackClose

    # sort to find top 10return stocks
    returns2 = sorted(returns.items(), key=lambda x: x[1], reverse=True)[:10]
    top_returns = {k[0]: returns[k[0]] for k in returns2}

    print("Evaluate 1Y since: ", period_start, top_returns.keys())
    eval_period_end = util.get_next(period_end, 3)
    if len(top_returns) == 10:
        r = stocks.get_portfolio_return(period_end, eval_period_end, get_portfolio(top_returns.keys()))
        print("backtest 1Q", period_end, eval_period_end, r)
        ByAdjClose.append(r.ByAdjClose)
        ByAddBackClose.append(r.ByAddBackClose)
    print("")

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

Evaluate 1Y since:  2000-01-01 dict_keys([])

Evaluate 1Y since:  2000-04-01 dict_keys(['L02', 'LJ3', 'H78', 'J36', 'C52', 'H02', 'U14', 'S20', 'S63', 'U11'])
backtest 1Q 2001-04-01 2001-07-01 ByAdjClose: -0.004094 ByAddBackClose: -0.006173

Evaluate 1Y since:  2000-07-01 dict_keys(['T14', 'LJ3', 'L02', 'G07', 'F13', 'H02', 'CTO', 'J36', 'BDA', 'BTJ'])
backtest 1Q 2001-07-01 2001-10-01 ByAdjClose: -1.164957 ByAddBackClose: -0.188339

Evaluate 1Y since:  2000-10-01 dict_keys(['T14', 'G07', 'D01', 'LJ3', 'H02', 'S20', 'F13', 'M01', 'L23', 'BAZ'])
backtest 1Q 2001-10-01 2002-01-01 ByAdjClose:  0.178842 ByAddBackClose:  0.178274

Evaluate 1Y since:  2001-01-01 dict_keys(['T14', 'G13', 'F13', 'D01', 'G07', 'BAZ', 'T15', 'S41', 'H02', 'V03'])
backtest 1Q 2002-01-01 2002-04-01 ByAdjClose:  0.095974 ByAddBackClose:  0.094782

Evaluate 1Y since:  2001-04-01 dict_keys(['T14', '600', 'AZR', 'F03', '578', 'G13', '554', 'F13', '504', 'S59'])
backtest 1Q 2002-04-01 2002-07-01 ByAdjClose: -0.047460 B

In [32]:
df = pd.DataFrame(ByAddBackClose, columns=['ByAddBackClose'])
df["log_return"] = np.log(df['ByAddBackClose'])
df["log_return"].sum()

-110.31067046971494

In [31]:
df = pd.DataFrame(ByAdjClose, columns=['ByAdjClose'])
df["log_return"] = np.log(df['ByAdjClose'])
df["log_return"].sum()

-104.24952276444728

# Manual portfolio

In [41]:
dy1 = ["M01", "5DD", "CC3", "i07", "BN4"]
dy2 = ["L38", "E5H", "1D0", "Z74", "CY6U"]
ww = ["A04", "5VP", "BQD", "F03", "NO4", "41B", "A34", "T14", "Q0X"]

period_start = util.YearMonth(2015, 1)
period_last = util.YearMonth(2023, 1)

portfolio = stocks.loads(ww)
while period_start <= period_last:
    period_end = util.get_next(period_start, 12)
    eval_period_end = util.get_next(period_end, 3)

    returns = stocks.get_returns(period_start, period_end, portfolio)
    w1 = stocks.get_weights_by_returns(returns)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio)
    print(eval_period_end,eval_period_end, r)

    r = stocks.get_portfolio_return(period_end, eval_period_end, portfolio, w1.ByAddBackClose)
    print(eval_period_end,eval_period_end, r)

    # roll to next quarter
    period_start = util.get_next(period_start, 3)

2016-04-01 2016-04-01 ByAdjClose: -0.048074 ByAddBackClose: -0.047998
2016-04-01 2016-04-01 ByAdjClose:  0.077765 ByAddBackClose:  0.077723
2016-07-01 2016-07-01 ByAdjClose: -0.030511 ByAddBackClose: -0.033910
2016-07-01 2016-07-01 ByAdjClose: -0.072943 ByAddBackClose: -0.073818
2016-10-01 2016-10-01 ByAdjClose: -0.023897 ByAddBackClose: -0.023897
2016-10-01 2016-10-01 ByAdjClose:  0.051323 ByAddBackClose:  0.051323
2017-01-01 2017-01-01 ByAdjClose:  0.136322 ByAddBackClose:  0.133236
2017-01-01 2017-01-01 ByAdjClose:  0.014725 ByAddBackClose:  0.010083
2017-04-01 2017-04-01 ByAdjClose:  0.149561 ByAddBackClose:  0.149536
2017-04-01 2017-04-01 ByAdjClose:  0.882782 ByAddBackClose:  0.883018
2017-07-01 2017-07-01 ByAdjClose:  0.009721 ByAddBackClose:  0.007242
2017-07-01 2017-07-01 ByAdjClose:  0.166543 ByAddBackClose:  0.162271
2017-10-01 2017-10-01 ByAdjClose: -0.022553 ByAddBackClose: -0.022552
2017-10-01 2017-10-01 ByAdjClose:  0.084065 ByAddBackClose:  0.084066
2018-01-01 2018-01-0

In [ ]:
# for i in range(2001, 2025):
#     portfolio = stocks.loads(ww)

#     start_day = str(i-1) + "-01-01"
#     end_day = str(i-1) + "-12-31"

#     returns = stocks.get_returns(start_day, end_day, portfolio)
#     w1 = stocks.get_weights_by_returns(returns)

#     start_day = str(i) + "-01-01"
#     end_day = str(i) + "-12-31"
#     r = stocks.get_portfolio_return(start_day, end_day, portfolio)
#     print(i, r)

#     r = stocks.get_portfolio_return(start_day, end_day, portfolio, w1.ByAdjClose)
#     print(i, r)
#     r = stocks.get_portfolio_return(start_day, end_day, portfolio, w1.ByAddBackClose)
#     print(i, r)

In [ ]:
# stock = stocks.load("D05")
# print(stock.get_sector())
df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
df.AvergeVolume = df.AvergeVolume / 10000
df = df[df.AvergeVolume>1].sort_values(by=['AvergeVolume'])

# df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
# df.MarketCap = df.MarketCap / 1000000
# df = df[df.MarketCap>1000].sort_values(by=['MarketCap'])

items = stocks.loads(df.Ticker.to_list())

# for i in range(2000, 2024):
#     start_day = str(i) + "-01-01"
#     end_day = str(i+1) + "-01-01"
#     # print(i, stocks.get_risk(start_day, end_day, stock))
#     print(i, stock.get_adtv(start_day, end_day))

for stock in items:
    print(stock)

# Adj Close return vs AddBack Return

In [183]:
# stock = stocks.load("D05")
stock = stocks.load("D8DU")
df = stocks.get_addback_close(stock, "2022-01-01", "2024-01-01")

return_adj_close = df["Adj Close"].pct_change().dropna()
return_addback_close = df["AddBackClose"].pct_change().dropna()

print(return_addback_close.mean())
print(return_adj_close.mean())
return_adj_close.std()  / return_addback_close.std() - 1
# return_adj_close.mean()  / return_addback_close.mean() - 1

-0.0002463114195503293
-0.00044110314428133615


0.04505055363062538

In [195]:
df = pd.concat([return_addback_close, return_adj_close], axis=1)
df.cov().iloc[0, 1]

0.000718800466679042

# Stock Market Cap Distribution

In [ ]:
df = util.to_df(info.get_all_mc(), ['Ticker', 'MarketCap'])
df.MarketCap = df.MarketCap / 1000000
plt = df[df.MarketCap<100].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - Below(100M SGD)')
plt.legend(['Total 365 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M.png')

plt = df[df.MarketCap>100][df.MarketCap<1500].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 100M~1.5B SGD)')
plt.legend(['Total 188 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_100M_1.5B.png')

plt = df[df.MarketCap>1500][df.MarketCap<30000].MarketCap.plot.hist(bins=50, alpha=0.5, title='SGX Stock Market Cap Distribution - 1.5B~30B SGD)')
plt.legend(['Total 62 stocks'])
plt.set_xlabel("Market Cap (1M)")
plt.get_figure().savefig('mc_1.5B_20B.png')

df[df.MarketCap>10000].sort_values(by=['MarketCap']).to_csv("mc_10B_plus.csv", index=False, float_format='%.6f')

# Average Daily Trading Volume Hist

In [ ]:
df = util.to_df(info.get_all_avg_vol(), ['Ticker', 'AvergeVolume'])
df.AvergeVolume = df.AvergeVolume / 10000

plt = df[df.AvergeVolume<=1].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) - Below(10K SGD)')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 365 stocks'])
plt.get_figure().savefig('adtv_10K.png')

plt = df[1<df.AvergeVolume][df.AvergeVolume<=30].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 10K~300K SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 207 stocks'])
plt.get_figure().savefig('adtv_10K_300K.png')

plt = df[df.AvergeVolume>30][df.AvergeVolume<=300].plot.hist(bins=50, alpha=0.5, title='SGX Average Daily Trading Volume (ADTV) 300K~3M SGD')
plt.set_xlabel("ADTV (10K)")
plt.legend(['ADTV Total 68 stocks'])
plt.get_figure().savefig('adtv_300K_3M.png')

df[df.AvergeVolume>300].sort_values(by=['AvergeVolume']).to_csv("adtv_3M_plus.csv", index=False, float_format='%.6f')

# STI Estimate

In [ ]:
es3 = stocks.load("ES3")
df = stocks.get_addback_close(es3, "2010-03-01", "2024-03-15")
df["Adj Close"] = df["Adj Close"] * 990
df["AddBackClose"] = df["AddBackClose"] * 1010
df["Close"] = df["Close"] * 1000

plt = df.plot(x="Day", y = ["AddBackClose", "Close", "Adj Close"], linewidth=0.8, figsize=(15, 10), title="Esitmated STI")
plt.legend(['Estimated STI if add back dividend', 'STI', 'Estimated STI if use Adj Close'])
plt.get_figure().savefig('STI_estimated.png')